In [1]:
import numpy as np

In [2]:
#   Tree cơ bản 
class SimpleRegressionTree:
    def __init__(self, max_depth=2, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None  

    def _mse(self, y):
        """Mean Squared Error của một tập y"""
        if len(y) == 0:
            return 0
        mean_y = np.mean(y)
        return np.mean((y - mean_y) ** 2)

    def _find_best_split(self, X, y):
        """Tìm feature và ngưỡng split tốt nhất theo giảm MSE"""
        n_samples, n_features = X.shape
        best_mse = float('inf')
        best_feature = None
        best_threshold = None
        best_left_idx = None
        best_right_idx = None

        current_mse = self._mse(y)

        for feature in range(n_features):
            # Lấy giá trị feature và sort
            vals = X[:, feature]
            sorted_idx = np.argsort(vals)
            sorted_X = vals[sorted_idx]
            sorted_y = y[sorted_idx]

            # Thử các ngưỡng giữa các giá trị khác nhau
            for i in range(1, n_samples):
                if sorted_X[i] == sorted_X[i-1]:
                    continue
                threshold = (sorted_X[i] + sorted_X[i-1]) / 2

                left_idx = sorted_idx[:i]
                right_idx = sorted_idx[i:]

                if len(left_idx) < self.min_samples_split or len(right_idx) < self.min_samples_split:
                    continue

                mse_left = self._mse(sorted_y[:i])
                mse_right = self._mse(sorted_y[i:])
                mse_split = (len(left_idx) * mse_left + len(right_idx) * mse_right) / n_samples

                if mse_split < best_mse:
                    best_mse = mse_split
                    best_feature = feature
                    best_threshold = threshold
                    best_left_idx = left_idx
                    best_right_idx = right_idx

        return best_feature, best_threshold, best_left_idx, best_right_idx

    def _build_tree(self, X, y, depth=0):
        if len(y) < self.min_samples_split or depth >= self.max_depth:
            return {"value": np.mean(y)}  # lá: trung bình y

        feature, threshold, left_idx, right_idx = self._find_best_split(X, y)

        if feature is None:
            return {"value": np.mean(y)}

        left_tree = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right_tree = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return {
            "feature": feature,
            "threshold": threshold,
            "left": left_tree,
            "right": right_tree
        }

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        return self

    def _predict_one(self, x, node):
        if "value" in node:
            return node["value"]
        if x[node["feature"]] <= node["threshold"]:
            return self._predict_one(x, node["left"])
        else:
            return self._predict_one(x, node["right"])

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])

In [3]:
#   Gradient Boosting Regression
class GradientBoostingRegressor:
    def __init__(self, n_estimators=30, learning_rate=0.1, max_depth=2, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
        self.initial_prediction = None
        self.trees = []
        self.tree_weights = []

    def fit(self, X, y):
        self.initial_prediction = np.mean(y)
        predictions = np.full_like(y, self.initial_prediction, dtype=float)
        
        print(f"Dự đoán ban đầu (mean của y): {self.initial_prediction:.2f}\n")

        for i in range(self.n_estimators):
            residuals = y - predictions

            tree = SimpleRegressionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split
            )
            tree.fit(X, residuals)

            update = tree.predict(X)

            predictions += self.learning_rate * update

            self.trees.append(tree)
            self.tree_weights.append(self.learning_rate)

            mse = np.mean((y - predictions) ** 2)
            mean_abs_res = np.mean(np.abs(residuals))
            print(f"Cây {i+1:2d} | MSE: {mse:.4f} | Mean |residual|: {mean_abs_res:.4f}")

        print("\nHuấn luyện hoàn tất.")
        return self

    def predict(self, X):
        pred = np.full(X.shape[0], self.initial_prediction, dtype=float)
        for tree, weight in zip(self.trees, self.tree_weights):
            pred += weight * tree.predict(X)
        return pred

In [4]:
#   Dữ liệu 
color_map = {'Hồng': 0, 'Xanh': 1, 'Đỏ': 2}
gender_map = {'Nam': 0, 'Nữ': 1}

X_full = np.array([
    [1.6, color_map['Hồng'], gender_map['Nam']],     # 88
    [1.6, color_map['Xanh'], gender_map['Nữ']],      # 76
    [1.5, color_map['Hồng'], gender_map['Nữ']],      # 56
    [1.8, color_map['Đỏ'],   gender_map['Nam']],     # 73
    [1.5, color_map['Hồng'], gender_map['Nam']],     # 77
    [1.4, color_map['Xanh'], gender_map['Nữ']],      # 57
])

y = np.array([88, 76, 56, 73, 77, 57])

print("Dữ liệu đầu vào (X_full):")
print("Cột 0: Chiều cao | Cột 1: Màu (0=Hồng,1=Xanh,2=Đỏ) | Cột 2: Giới tính (0=Nam,1=Nữ)")
print(X_full)
print("\nTarget y:", y, "\n")

Dữ liệu đầu vào (X_full):
Cột 0: Chiều cao | Cột 1: Màu (0=Hồng,1=Xanh,2=Đỏ) | Cột 2: Giới tính (0=Nam,1=Nữ)
[[1.6 0.  0. ]
 [1.6 1.  1. ]
 [1.5 0.  1. ]
 [1.8 2.  0. ]
 [1.5 0.  0. ]
 [1.4 1.  1. ]]

Target y: [88 76 56 73 77 57] 



In [5]:
#   Huấn luyện và dự đoán
gb = GradientBoostingRegressor(
    n_estimators=25,          # đủ để hội tụ trên data nhỏ
    learning_rate=0.1,
    max_depth=2,              # cây nông 
    min_samples_split=2
)

gb.fit(X_full, y)

# Dự đoán lại trên tập huấn luyện để kiểm tra
y_pred = gb.predict(X_full)

print("\nKết quả dự đoán so với thực tế:")
print("  Chiều cao | Màu | Giới | Thực tế | Dự đoán | Residual")
for i in range(len(y)):
    h = X_full[i, 0]
    c = {0:'Hồng', 1:'Xanh', 2:'Đỏ'}[int(X_full[i, 1])]
    g = {0:'Nam', 1:'Nữ'}[int(X_full[i, 2])]
    print(f"  {h:.1f}      | {c:4} | {g:4} | {y[i]:7.0f} | {y_pred[i]:7.2f} | {y[i] - y_pred[i]:8.2f}")

print(f"\nMSE cuối cùng: {np.mean((y - y_pred)**2):.4f}")


Dự đoán ban đầu (mean của y): 71.17

Cây  1 | MSE: 116.4669 | Mean |residual|: 9.7778
Cây  2 | MSE: 105.6046 | Mean |residual|: 9.2333
Cây  3 | MSE: 96.0322 | Mean |residual|: 8.7293
Cây  4 | MSE: 87.8348 | Mean |residual|: 8.4058
Cây  5 | MSE: 80.6036 | Mean |residual|: 8.1869
Cây  6 | MSE: 74.4176 | Mean |residual|: 7.9812
Cây  7 | MSE: 68.9547 | Mean |residual|: 7.7910
Cây  8 | MSE: 64.2868 | Mean |residual|: 7.6123
Cây  9 | MSE: 60.1597 | Mean |residual|: 7.4471
Cây 10 | MSE: 56.6375 | Mean |residual|: 7.2917
Cây 11 | MSE: 53.5193 | Mean |residual|: 7.1482
Cây 12 | MSE: 50.8617 | Mean |residual|: 7.0132
Cây 13 | MSE: 48.5058 | Mean |residual|: 6.8885
Cây 14 | MSE: 46.5006 | Mean |residual|: 6.7711
Cây 15 | MSE: 44.7205 | Mean |residual|: 6.6628
Cây 16 | MSE: 43.2077 | Mean |residual|: 6.5608
Cây 17 | MSE: 41.8626 | Mean |residual|: 6.4667
Cây 18 | MSE: 40.7213 | Mean |residual|: 6.3780
Cây 19 | MSE: 39.6388 | Mean |residual|: 6.2964
Cây 20 | MSE: 38.6465 | Mean |residual|: 6.2168
C

In [7]:
# Dữ liệu mới - định dạng giống hệt: [chiều cao, màu mã hóa, giới tính mã hóa]
X_new = np.array([
    [1.70, color_map['Hồng'], gender_map['Nam']], 
    [1.55, color_map['Xanh'], gender_map['Nữ']],     
    [1.65, color_map['Đỏ'],   gender_map['Nam']],    
    [1.45, color_map['Hồng'], gender_map['Nữ']],     
    [1.75, color_map['Xanh'], gender_map['Nam']],    
])


y_new_pred = gb.predict(X_new)
print("\n" + "="*60)
print("DỰ ĐOÁN TRÊN DỮ LIỆU MỚI (test set)")
print("="*60)
print("  Chiều cao | Màu   | Giới | Dự đoán cân nặng (kg)")
print("-"*60)
for i in range(len(X_new)):
    h = X_new[i, 0]
    c_name = {v: k for k, v in color_map.items()}[int(X_new[i, 1])]
    g_name = {v: k for k, v in gender_map.items()}[int(X_new[i, 2])]
    print(f"  {h:.2f}      | {c_name:5} | {g_name:4} | {y_new_pred[i]:.2f}")
print("="*60)


DỰ ĐOÁN TRÊN DỮ LIỆU MỚI (test set)
  Chiều cao | Màu   | Giới | Dự đoán cân nặng (kg)
------------------------------------------------------------
  1.70      | Hồng  | Nam  | 81.28
  1.55      | Xanh  | Nữ   | 61.05
  1.65      | Đỏ    | Nam  | 79.56
  1.45      | Hồng  | Nữ   | 62.78
  1.75      | Xanh  | Nam  | 79.56
